## Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Etapas da atividade:

### Etapa 1: Carregamento e inspeção inicial

In [2]:
df_initial = pd.read_csv("./data/df_full.csv")

df = df_initial.copy()

df.head()

,Timestamp,S1_1,S1_2,S1_3,S1_4,S1_5,S1_6,S2_1,S2_2,S3_1,S3_2,S4_1,S4_2,Nivel_agua,Nivel_total
0,28/03/2022 16:30:26.57745,1524.09200,1524.68570,1530.24903,1538.64225,1552.56175,1553.28008,1530.97697,1539.25600,1544.74917,1550.44190,1545.14272,1559.75003,257.0,257.0
1,28/03/2022 16:30:27.07745,1524.09059,1524.68520,1530.24959,1538.64228,1552.56153,1553.28018,1530.97808,1539.25592,1544.75001,1550.44253,1545.14266,1559.75004,257.0,257.0
2,28/03/2022 16:30:27.57745,1524.09108,1524.68440,1530.24883,1538.64254,1552.56085,1553.28046,1530.97912,1539.25621,1544.74792,1550.44108,1545.14126,1559.75247,257.0,257.0
3,28/03/2022 16:30:28.07745,1524.09161,1524.68487,1530.24907,1538.64199,1552.56206,1553.28030,1530.97905,1539.25607,1544.74877,1550.44254,1545.14166,1559.75295,257.0,257.0
4,28/03/2022 16:30:28.57745,1524.09070,1524.68511,1530.24922,1538.64187,1552.56135,1553.27996,1530.97980,1539.25519,1544.74875,1550.44097,1545.14139,1559.74994,257.0,257.0


In [3]:
df.shape

(66540, 15)

In [4]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66540 entries, 0 to 66539
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Timestamp    66540 non-null  object 
 1   S1_1         66540 non-null  float64
 2   S1_2         66540 non-null  float64
 3   S1_3         66540 non-null  float64
 4   S1_4         66540 non-null  float64
 5   S1_5         66540 non-null  float64
 6   S1_6         66540 non-null  float64
 7   S2_1         66540 non-null  float64
 8   S2_2         66540 non-null  float64
 9   S3_1         66540 non-null  float64
 10  S3_2         66540 non-null  float64
 11  S4_1         66540 non-null  float64
 12  S4_2         66540 non-null  float64
 13  Nivel_agua   66540 non-null  float64
 14  Nivel_total  66540 non-null  float64
dtypes: float64(14), object(1)
memory usage: 7.6+ MB


### Sumário

- Timestamp: Refere-se ao momento em que uma medição foi registrada.
- S1_1 a S1_6: São os nomes de um conjunto de seis sensores de temperatura baseados em Fibra com Rede de Bragg (FBG). Os valores associados a eles são as medições de comprimento de onda em nanômetros (nm).
- S2_1, S2_2, S3_1, S3_2, S4_1, S4_2: São os nomes de seis sensores de pressão hidrostática, também baseados em FBG, distribuídos em três canais (2, 3 e 4), com dois sensores por canal. Os valores representam as medições de comprimento de onda (em nm) desses sensores.
- Nivel agua: Representa a medição do nível da interface água-óleo, ou seja, a altura da camada de água no fundo do tanque, expressa em milímetros (mm).
- Nivel total: Representa a medição do nível total de líquido (água + óleo) dentro do tanque, também expressa em milímetros (mm). A camada de óleo pode ser calculada subtraindo o nível da água do nível total.

### Etapa 2: Resumo estatístico

In [5]:
df.describe()

,S1_1,S1_2,S1_3,S1_4,S1_5,S1_6,S2_1,S2_2,S3_1,S3_2,S4_1,S4_2,Nivel_agua,Nivel_total
count,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000,66540.000000
mean,1524.134287,1524.730478,1530.291499,1538.683594,1552.585198,1553.321983,1531.088692,1539.395152,1544.837456,1550.519120,1545.308524,1558.082188,1245.555020,1596.034836
std,0.054754,0.055615,0.056080,0.065582,0.060931,0.076920,0.166813,0.158077,0.110521,0.150314,0.151640,4.585039,734.131379,745.587549
min,1523.876580,1524.632450,1530.188590,1538.571820,1552.386530,1553.097760,1530.652050,1539.029490,1544.519990,1550.201170,1545.027500,1545.204630,109.000000,109.000000
25%,1524.089350,1524.688170,1530.248818,1538.641880,1552.556660,1553.277568,1531.038010,1539.360270,1544.746688,1550.397470,1545.222540,1559.539840,791.000000,1287.000000
50%,1524.128870,1524.720210,1530.281990,1538.672615,1552.571130,1553.290640,1531.080450,1539.386375,1544.843610,1550.510815,1545.287020,1559.679345,1219.000000,1656.000000
75%,1524.160922,1524.765310,1530.314970,1538.705990,1552.584570,1553.333610,1531.185390,1539.425900,1544.924012,1550.622420,1545.383850,1559.762980,1880.000000,2188.000000
max,1524.299000,1524.905580,1530.464740,1538.917920,1552.823080,1553.571740,1531.635210,1540.003880,1545.096100,1550.912600,1545.873190,1560.415660,2804.000000,2804.000000


### Etapa 3: Valores ausentes e duplicados

In [6]:
df.isna().sum()


Timestamp      0
S1_1           0
S1_2           0
S1_3           0
S1_4           0
S1_5           0
S1_6           0
S2_1           0
S2_2           0
S3_1           0
S3_2           0
S4_1           0
S4_2           0
Nivel_agua     0
Nivel_total    0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

### Etapa 4: Distribuição das variáveis

### Etapa 5: Correlação entre sensores

### Etapa 6: Análise temporal

### Etapa 7: Relação com as variáveis-alvo

## Resumo final